<h2>Capstone Project_Week 4/5: Let’s do ANY CITY in 3 days!

### 1. Importing the libraries

In [310]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_rows",50, "display.max_columns", None, "display.max_colwidth", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 2. Determine geographic coordinates of target city

In [353]:
address = 'Lübeck, Germany'

geolocator = Nominatim(user_agent="city_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinates of Lübeck, Germany are 53.866444, 10.684738.


### 3. Retrieve venues in target city from Foursquare.com

The objective of this project is to generate a unique itinerary for each of the 3 days of the trip to the target city.

Each day should include the following 5 items:
1. An area with many breakfast venues ("breakfast area")
2. One of the 3 top rated museums ("top museum")
3. One of the 3 top rated sights ("top sight")
4. An area with many restaurants ("dining area")
5. An areas with many bars and clubs ("nightlife area")


#### 3.1 Search for areas with top rated breakfast venues

##### 3.1.1 Retrieve all breakfast venues of target city as a dataframe

In [354]:
# Foursquare.com credentials and parameters
CLIENT_ID = 'XYZ' # your Foursquare ID
CLIENT_SECRET = 'XYZ' # your Foursquare Secret
VERSION = '20201110' # Foursquare API version

categoryId='4bf58dd8d48988d179941735,4bf58dd8d48988d143941735,4bf58dd8d48988d16d941735,4bf58dd8d48988d1e0931735,4bf58dd8d48988d147941735,4bf58dd8d48988d148941735'

#Bagel Shop
#4bf58dd8d48988d179941735
#Breakfast Spot
#4bf58dd8d48988d143941735
#Café
#4bf58dd8d48988d16d941735
#Coffee Shop
#4bf58dd8d48988d1e0931735
#Diner
#4bf58dd8d48988d147941735
#Donut Shop
#4bf58dd8d48988d148941735

breakfast_category_name_exclude=[]

radius=5000
LIMIT = 100 # A default Foursquare API limit value

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            latitude, 
            longitude, 
            VERSION, 
            radius,
            LIMIT,
            categoryId)
            
# make the GET request
breakfast_json = requests.get(url).json()['response']['venues']
       
# return only relevant information for each nearby venue
breakfast_list=[]


for v in breakfast_json:
        try:
            venue_name=v['name']
        except:
            venue_name='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            venue_lng=v['location']['lng']
        except:
            venue_lng='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            category_name=v['categories'][0]['name']
        except:
            category_name='no data'
        try:
            venue_id=v['id']
        except:
            venue_id='no data'
        
        breakfast_list.append([(
            venue_name, 
            venue_lat, 
            venue_lng, 
            category_name,
            venue_id)])

if len(breakfast_list)==0:
    print('No breakfast venues found within radius of {}m of {}'.format(radius, address))

else:
        breakfast_df = pd.DataFrame([item for venue_list in breakfast_list for item in venue_list])
        breakfast_df.columns = ['Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category','Venue ID']
        breakfast_df=breakfast_df[breakfast_df['Venue Category'].isin(breakfast_category_name_exclude)==False]
        breakfast_df.loc[:,'Venue Key']=range(len(breakfast_df.Venue))
        breakfast_df.reset_index(drop=True, inplace=True)
        print('Number of breakfast venues in {} (latitude: {},longitude: {}): {}\n'.format(address,latitude,longitude,breakfast_df.shape[0]))
        breakfast_df

Number of breakfast venues in Lübeck, Germany (latitude: 53.866444,longitude: 10.684738): 50



In [355]:
breakfast_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key
0,Ratskeller Zu Lübeck,53.866889,10.685227,German Restaurant,4c6bdb9ba48420a15cd70a0b,0
1,Niederegger Arcadencafé,53.866597,10.685088,Café,4c67d32119f3c9b6cfb0a3ff,1
2,Niederegger Stammhaus,53.866519,10.685618,Candy Store,4c26147d5c5ca593196b45fe,2
3,Conditorei & Café Maret,53.866811,10.684817,Café,4db2b58f1e7248d1357b89da,3
4,Tchibo,53.867090,10.685804,Coffee Shop,4d280bdffb8e5941e9584d54,4


#### 3.1.2 Add breakfast venues' ratings

In [356]:
# retrieve ratings of breakfast venues and append to breakfast_df (Attention: venue/rating is a premium call!)
venue_id=breakfast_df.loc[:,'Venue ID']
breakfast_df.loc[:,'Venue Rating']=0

for i in range(len(venue_id)):
#    print('\nvenue_id [',i,']:',venue_id[i])
    venue_var=venue_id[i]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_var, CLIENT_ID, CLIENT_SECRET, VERSION)
    try:
#        print('Venue rating: ', requests.get(url).json()['response']['venue']['rating'])
        breakfast_df.loc[i,'Venue Rating']=requests.get(url).json()['response']['venue']['rating']
    except:
#        print('Venue rating: This venue has not been rated yet.')
        breakfast_df.loc[i,'Venue Rating']='no rating'

breakfast_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating
0,Ratskeller Zu Lübeck,53.866889,10.685227,German Restaurant,4c6bdb9ba48420a15cd70a0b,0,no rating
1,Niederegger Arcadencafé,53.866597,10.685088,Café,4c67d32119f3c9b6cfb0a3ff,1,no rating
2,Niederegger Stammhaus,53.866519,10.685618,Candy Store,4c26147d5c5ca593196b45fe,2,no rating
3,Conditorei & Café Maret,53.866811,10.684817,Café,4db2b58f1e7248d1357b89da,3,no rating
4,Tchibo,53.867090,10.685804,Coffee Shop,4d280bdffb8e5941e9584d54,4,no rating


#### 3.1.3 Show top 6 rated breakfast venues (breakfast venues without rating dropped)

In [315]:
top_breakfast_df=breakfast_df.loc[((breakfast_df['Venue Rating']!='no rating')&(breakfast_df['Venue Category'].str.contains('estaurant')==False)&(breakfast_df['Venue Category'].str.contains('tore')==False)&(breakfast_df['Venue Category'].str.contains('akery')==False)),:].sort_values('Venue Rating', ascending=False)
top_breakfast_df.reset_index(drop=True, inplace=True)
top_breakfast_df=top_breakfast_df.head(6)
top_breakfast_df

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating
0,Luicella's,53.559663,10.014307,Ice Cream Shop,56e43e5e498e0ee2d01e7aab,21,9.2
1,Playground Coffee,53.551626,9.965190,Coffee Shop,58b4918614f8f455a15556b6,19,9.2
2,Elbgold,53.583771,9.983932,Coffee Shop,549561d9498e2fe76c0bd932,26,9.1
3,Hermetic Coffee Roasters,53.559736,9.966826,Coffee Shop,50ae4f74f136df30293d9438,16,9.1
4,Public Coffee Roasters,53.551372,9.982597,Coffee Shop,52e12f23498eab3870ffa96c,22,9
5,Elbgold,53.563152,9.967200,Coffee Roaster,4c61ce87edb29c741faf20a7,17,9


#### 3.1.4 Determine 6 breakfast areas (breakfast venues clustered based on coordinates)

3.1.4.1 Run _k_-means to cluster the breakfast venues into 6 clusters.


In [316]:
# set number of clusters
breakfast_kclusters = 6

breakfast_areas = breakfast_df.drop(['Venue','Venue Category', 'Venue ID', 'Venue Rating','Venue Key'], 1)

# run k-means clustering
breakfast_kmeans = KMeans(n_clusters=breakfast_kclusters, random_state=0).fit(breakfast_areas)

# check cluster labels generated for each row in the dataframe
breakfast_kmeans.labels_[0:10] 

array([0, 0, 0, 4, 0, 0, 4, 4, 0, 0], dtype=int32)

In [317]:
breakfast_areas.head()

,Venue Latitude,Venue Longitude
0,53.551256,10.000930
1,53.549879,10.001518
2,53.553136,10.001385
3,53.556275,9.963038
4,53.546953,9.997440


3.1.4.2 Create a dataframe that includes the 6 breakfast areas and the breakfast venues in each breakfast area.


In [318]:
# add clustering labels
try:
    breakfast_areas.insert(0, 'Item Key', breakfast_kmeans.labels_)
    breakfast_areas['Venue Key']=breakfast_df['Venue Key']
    breakfast_areas_merged = breakfast_df
except:
    breakfast_areas_merged = breakfast_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
breakfast_areas_merged = breakfast_areas_merged.join(breakfast_areas[['Venue Key','Item Key']].set_index('Venue Key'), on='Venue Key')

breakfast_areas_merged.head() # check the last columns!

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating,Item Key
0,Starbucks,53.551256,10.000930,Coffee Shop,4b06afd3f964a520a3ee22e3,0,6.8,0
1,Die Rösterei,53.549879,10.001518,Coffee Shop,55ed6855498ea54db8a39afe,1,8.5,0
2,Enjoy Coffee Bar,53.553136,10.001385,Café,520de75b11d2a9d3ac79f1ee,2,7.6,0
3,Kante,53.556275,9.963038,Café,5ae58d3d25fb7b002cdc0742,3,no rating,4
4,Public Coffee Roasters,53.546953,9.997440,Coffee Shop,59aaaf194a1cc02d7a736c6a,4,8.7,0


3.1.4.3 Determine the centroid coordinates for each breakfast area

In [319]:
# Centroid coordinates of breakfast areas
for i in breakfast_areas_merged.index:
    breakfast_areas_merged.loc[i,'Item Latitude']=breakfast_kmeans.cluster_centers_[breakfast_areas_merged['Item Key'][i]][0]
    breakfast_areas_merged.loc[i,'Item Longitude']=breakfast_kmeans.cluster_centers_[breakfast_areas_merged['Item Key'][i]][1]
breakfast_areas_merged.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating,Item Key,Item Latitude,Item Longitude
0,Starbucks,53.551256,10.000930,Coffee Shop,4b06afd3f964a520a3ee22e3,0,6.8,0,53.551651,9.99362
1,Die Rösterei,53.549879,10.001518,Coffee Shop,55ed6855498ea54db8a39afe,1,8.5,0,53.551651,9.99362
2,Enjoy Coffee Bar,53.553136,10.001385,Café,520de75b11d2a9d3ac79f1ee,2,7.6,0,53.551651,9.99362
3,Kante,53.556275,9.963038,Café,5ae58d3d25fb7b002cdc0742,3,no rating,4,53.558838,9.96683
4,Public Coffee Roasters,53.546953,9.997440,Coffee Shop,59aaaf194a1cc02d7a736c6a,4,8.7,0,53.551651,9.99362


3.1.4.4 Show the 6 breakfast areas with their geographic coordinates

In [320]:
top_breakfast_areas=breakfast_areas_merged.loc[:,['Item Key', 'Item Latitude','Item Longitude']].groupby('Item Key', as_index=False).mean()
top_breakfast_areas.loc[:,'Item Category']='Breakfast Area'
for i in top_breakfast_areas.loc[:,'Item Key']:
    top_breakfast_areas.loc[i,'Item Name']='Breakfast Area '+str(top_breakfast_areas.loc[i,'Item Key'])
top_breakfast_areas

,Item Key,Item Latitude,Item Longitude,Item Category,Item Name
0,0,53.551651,9.993620,Breakfast Area,Breakfast Area 0
1,1,53.563466,9.941182,Breakfast Area,Breakfast Area 1
2,2,53.549519,10.037344,Breakfast Area,Breakfast Area 2
3,3,53.573069,10.004401,Breakfast Area,Breakfast Area 3
4,4,53.558838,9.966830,Breakfast Area,Breakfast Area 4
5,5,53.505420,9.986052,Breakfast Area,Breakfast Area 5


#### 3.2 Search for top rated museums

##### 3.2.1 Retrieve all museums of target city as a dataframe

In [321]:
# Foursquare.com credentials and parameters
CLIENT_ID = 'R2B3BSMOAZU414BSOGLAYMB2MJPFUDT1PPEL1M0KZMD4RJFK' # your Foursquare ID
CLIENT_SECRET = '1YM0O41P05UKBSIDUTBL1G4ZYCAKEVRNQ4VFIO40THDU3EWW' # your Foursquare Secret
VERSION = '20201110' # Foursquare API version
categoryId='4bf58dd8d48988d181941735' #venue category: museum: 4bf58dd8d48988d181941735
radius=5000 # search radius around address coordinates (in m)
LIMIT = 100 # A default Foursquare API limit value

museums_category_name_exclude=[]

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            latitude, 
            longitude, 
            VERSION, 
            radius,
            LIMIT,
            categoryId)
            
# make the GET request
museums_json = requests.get(url).json()['response']['venues']
       
# return only relevant information for each nearby venue
museums_list=[]

for v in museums_json:
        try:
            venue_name=v['name']
        except:
            venue_name='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            venue_lng=v['location']['lng']
        except:
            venue_lng='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            category_name=v['categories'][0]['name']
        except:
            category_name='no data'
        try:
            venue_id=v['id']
        except:
            venue_id='no data'
        
        museums_list.append([(
            venue_name, 
            venue_lat, 
            venue_lng, 
            category_name,
            venue_id)])

if len(museums_list)==0:
    print('No museums found within radius of {}m of {}'.format(radius, address))

else:
        museums_df = pd.DataFrame([item for venue_list in museums_list for item in venue_list])
        museums_df.columns = ['Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category','Venue ID']
        museums_df=museums_df[museums_df['Venue Category'].isin(museums_category_name_exclude)==False]
        museums_df.loc[:,'Venue Key']=range(len(museums_df.Venue))
        museums_df.reset_index(drop=True, inplace=True)
        print('Number of museums in {} (latitude: {},longitude: {}): {}\n'.format(address,latitude,longitude,museums_df.shape[0]))
        museums_df

Number of museums in Hamburg, Germany (latitude: 53.550341,longitude: 10.000654): 49



In [322]:
museums_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key
0,Chocoversum,53.548019,10.002306,Museum,4e9bf330f9f42a0307fa0b90,0
1,Museum für Kunst und Gewerbe,53.551453,10.009743,Art Museum,4b05885ff964a5207ac122e3,1
2,Hamburger Kunsthalle - Galerie der Gegenwart,53.555376,10.002739,Art Museum,4b05885ff964a5207cc122e3,2
3,Museum der Illusionen,53.553269,10.003322,Museum,5bd1d0c9d1a402002ccf19bb,3
4,KomponistenQuartier,53.551370,9.977123,Museum,58e380245a58694e398d759e,4


##### 3.2.2 Add museums' ratings

In [323]:
# retrieve ratings of museums and append to museums_df (Attention: venue/rating is a premium call!)
venue_id=museums_df.loc[:,'Venue ID']
museums_df.loc[:,'Venue Rating']=0

for i in range(len(venue_id)):
#    print('\nvenue_id [',i,']:',venue_id[i])
    venue_var=venue_id[i]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_var, CLIENT_ID, CLIENT_SECRET, VERSION)
    try:
#        print('Venue rating: ', requests.get(url).json()['response']['venue']['rating'])
        museums_df.loc[i,'Venue Rating']=requests.get(url).json()['response']['venue']['rating']
    except:
#        print('Venue rating: This venue has not been rated yet.')
        museums_df.loc[i,'Venue Rating']='no rating'

museums_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating
0,Chocoversum,53.548019,10.002306,Museum,4e9bf330f9f42a0307fa0b90,0,no rating
1,Museum für Kunst und Gewerbe,53.551453,10.009743,Art Museum,4b05885ff964a5207ac122e3,1,no rating
2,Hamburger Kunsthalle - Galerie der Gegenwart,53.555376,10.002739,Art Museum,4b05885ff964a5207cc122e3,2,no rating
3,Museum der Illusionen,53.553269,10.003322,Museum,5bd1d0c9d1a402002ccf19bb,3,no rating
4,KomponistenQuartier,53.551370,9.977123,Museum,58e380245a58694e398d759e,4,no rating


##### 3.2.3 Show top 6 rated museums (museums without rating dropped)

In [324]:
top_museums_df=museums_df.loc[((museums_df['Venue Rating']!='no rating')& ((museums_df['Venue Category'].str.contains('useum')))),:].sort_values('Venue Rating', ascending=False)
top_museums_df.loc[:,['Item Key','Item Name','Item Latitude','Item Longitude','Item Category']]=[top_museums_df.loc[:,'Venue Key'],top_museums_df.loc[:,'Venue'],top_museums_df.loc[:,'Venue Latitude'],top_museums_df.loc[:,'Venue Longitude'],'Top Museums']
top_museums_df.reset_index(drop=True, inplace=True)
top_museums_df=top_museums_df.head(6)
top_museums_df

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating,Item Key,Item Name,Item Latitude,Item Longitude,Item Category


#### 3.3 Search for top rated sights

##### 3.3.1 Retrieve all sights of target city as a dataframe

In [325]:
# Foursquare.com credentials and parameters
CLIENT_ID = 'R2B3BSMOAZU414BSOGLAYMB2MJPFUDT1PPEL1M0KZMD4RJFK' # your Foursquare ID
CLIENT_SECRET = '1YM0O41P05UKBSIDUTBL1G4ZYCAKEVRNQ4VFIO40THDU3EWW' # your Foursquare Secret
VERSION = '20201110' # Foursquare API version

categoryId='4deefb944765f83613cdba6e,5642206c498e4bfca532186c,4bf58dd8d48988d131941735,4bf58dd8d48988d126941735,56aa371be4b08b9a8d573544,4bf58dd8d48988d1e2941735,52e81612bcbc57f1066b7a22,4bf58dd8d48988d1df941735,50aaa49e4b90af0d42d5de11,56aa371be4b08b9a8d573511,56aa371be4b08b9a8d573547,4bf58dd8d48988d1e0941735,4bf58dd8d48988d15d941735,52e81612bcbc57f1066b7a14,52e81612bcbc57f1066b7a25,4bf58dd8d48988d165941735,5032848691d4c4b30a586d61,56aa371be4b08b9a8d573560,56aa371be4b08b9a8d5734c3,5bae9231bedf3950379f89c7'

#4deefb944765f83613cdba6e #Historic Site
#5642206c498e4bfca532186c #Memorial Site
#4bf58dd8d48988d131941735 #Spiritual Center (church, mosque, monastery, temple etc.)
#4bf58dd8d48988d126941735 #Government Building
#56aa371be4b08b9a8d573544 #Bay
#4bf58dd8d48988d1e2941735 #Beach
#4bf58dd8d48988d1df941735 #Bridge
#50aaa49e4b90af0d42d5de11 #Castle
#56aa371be4b08b9a8d573511 #Cave
#56aa371be4b08b9a8d573547 #Fountain
#4bf58dd8d48988d1e0941735 #Harbor / Marina
#4bf58dd8d48988d15d941735 #Lighthouse
#52e81612bcbc57f1066b7a14 #Palace
#52e81612bcbc57f1066b7a25 #Pedestrian Plaza
#4bf58dd8d48988d165941735 #Scenic Lookout
#56aa371be4b08b9a8d5734c3 #Waterfront
#5bae9231bedf3950379f89c7 #Windmill

sights_category_name_exclude=[]

radius=5000
LIMIT = 100 # A default Foursquare API limit value

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            latitude, 
            longitude, 
            VERSION, 
            radius,
            LIMIT,
            categoryId)
            
# make the GET request
sights_json = requests.get(url).json()['response']['venues']
       
# return only relevant information for each nearby venue
sights_list=[]


for v in sights_json:
        try:
            venue_name=v['name']
        except:
            venue_name='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            venue_lng=v['location']['lng']
        except:
            venue_lng='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            category_name=v['categories'][0]['name']
        except:
            category_name='no data'
        try:
            venue_id=v['id']
        except:
            venue_id='no data'
        
        sights_list.append([(
            venue_name, 
            venue_lat, 
            venue_lng, 
            category_name,
            venue_id)])

if len(sights_list)==0:
    print('No sights found within radius of {}m of {}'.format(radius, address))

else:
        sights_df = pd.DataFrame([item for venue_list in sights_list for item in venue_list])
        sights_df.columns = ['Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Name','Venue ID']
        sights_df=sights_df[sights_df['Venue Name'].isin(sights_category_name_exclude)==False]
        sights_df.loc[:,'Venue Key']=range(len(sights_df.Venue))
        sights_df.reset_index(drop=True, inplace=True)
        print('Number of sights in {} (latitude: {},longitude: {}): {}\n'.format(address,latitude,longitude,sights_df.shape[0]))
        sights_df

Number of sights in Hamburg, Germany (latitude: 53.550341,longitude: 10.000654): 48



In [326]:
sights_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Name,Venue ID,Venue Key
0,Kontorhausviertel,53.549236,10.002703,Historic Site,5160175fe4b0137131095043,0
1,Chilehaus,53.548350,10.001947,Building,4b05885cf964a520bfc022e3,1
2,Binnenalster,53.555090,9.995134,Lake,4b92affff964a5208e1134e3,2
3,Jungfernstieg,53.552862,9.993174,Plaza,4ba67fa6f964a520615839e3,3
4,Hamburger Rathaus,53.550444,9.992430,City Hall,4b05885df964a520e8c022e3,4


##### 3.3.2 Add sights' ratings

In [327]:
# retrieve ratings of museums and append to museums_df (Attention: venue/rating is a premium call!)
venue_id=sights_df.loc[:,'Venue ID']
sights_df.loc[:,'Venue Rating']=0

for i in range(len(venue_id)):
#    print('\nvenue_id [',i,']:',venue_id[i])
    venue_var=venue_id[i]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_var, CLIENT_ID, CLIENT_SECRET, VERSION)
    try:
#        print('Venue rating: ', requests.get(url).json()['response']['venue']['rating'])
        sights_df.loc[i,'Venue Rating']=requests.get(url).json()['response']['venue']['rating']
    except:
#        print('Venue rating: This venue has not been rated yet.')
        sights_df.loc[i,'Venue Rating']='no rating'

sights_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Name,Venue ID,Venue Key,Venue Rating
0,Kontorhausviertel,53.549236,10.002703,Historic Site,5160175fe4b0137131095043,0,no rating
1,Chilehaus,53.548350,10.001947,Building,4b05885cf964a520bfc022e3,1,no rating
2,Binnenalster,53.555090,9.995134,Lake,4b92affff964a5208e1134e3,2,no rating
3,Jungfernstieg,53.552862,9.993174,Plaza,4ba67fa6f964a520615839e3,3,no rating
4,Hamburger Rathaus,53.550444,9.992430,City Hall,4b05885df964a520e8c022e3,4,no rating


##### 3.3.3 Show top 6 rated sights (sights without rating dropped)

In [328]:
#top_sights_df=sights_df.loc[((sights_df['Venue Rating']!='no rating')&(sights_df['Venue Name'].str.contains('useum'))),:].sort_values('Venue Rating', ascending=False)
top_sights_df=sights_df.loc[((sights_df['Venue Rating']!='no rating')),:].sort_values('Venue Rating', ascending=False)
top_sights_df.loc[:,['Item Key','Item Name','Item Latitude','Item Longitude','Item Category']]=[top_sights_df.loc[:,'Venue Key'],top_sights_df.loc[:,'Venue'],top_sights_df.loc[:,'Venue Latitude'],top_sights_df.loc[:,'Venue Longitude'],'Top Sights']
top_sights_df.reset_index(drop=True, inplace=True)
top_sights_df=top_sights_df.head(6)
top_sights_df

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,Venue,Venue Latitude,Venue Longitude,Venue Name,Venue ID,Venue Key,Venue Rating,Item Key,Item Name,Item Latitude,Item Longitude,Item Category


#### 3.4 Search for areas with many restaurants

##### 3.4.1 Retrieve all restaurants of target city as a dataframe

In [329]:
# Foursquare.com credentials and parameters
CLIENT_ID = 'R2B3BSMOAZU414BSOGLAYMB2MJPFUDT1PPEL1M0KZMD4RJFK' # your Foursquare ID
CLIENT_SECRET = '1YM0O41P05UKBSIDUTBL1G4ZYCAKEVRNQ4VFIO40THDU3EWW' # your Foursquare Secret
VERSION = '20201110' # Foursquare API version

categoryId='4d4b7105d754a06374d81259'

#Food
#4d4b7105d754a06374d81259

restaurants_category_exclude=['Bagel Shop','Bakery','Breakfast Spot','Bubble Tea Shop','Cafeteria','Café','Candy Store','Coffee Shop','Deli / Bodega','Dessert Shop','Diner','Donut Shop','Fast Food Restaurant','Fish & Chips Shop','Food & Drink Shop',
                       'Food Stand','Food Truck','Fried Chicken Joint','Ice Cream Shop','Hotel','Rest Area',
                       'Friterie','Gastropub','Hot Dog Joint','Irish Pub','Juice Bar','Pet Café','Pizza Place','Snack Place','Soup Place','Tea Room','Wings Joint','Bagel Shop','Bakery','Breakfast Spot','Bubble Tea Shop','Cafeteria',
                       'Café','Coffee Shop','Deli / Bodega','Dessert Shop','Diner','Donut Shop','Fast Food Restaurant','Fish & Chips Shop','Food Stand','Food Truck','Fried Chicken Joint','Friterie','Gastropub','Hot Dog Joint',
                       'Irish Pub','Juice Bar','Pet Café','Pizza Place','Snack Place','Soup Place','Tea Room','Wings Joint','Market','Pastry Shop']

radius=5000
LIMIT = 100 # A default Foursquare API limit value

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            latitude, 
            longitude, 
            VERSION, 
            radius,
            LIMIT,
            categoryId)
            
# make the GET request
restaurants_json = requests.get(url).json()['response']['venues']
       
# return only relevant information for each nearby venue
restaurants_list=[]


for v in restaurants_json:
        try:
            venue_name=v['name']
        except:
            venue_name='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            venue_lng=v['location']['lng']
        except:
            venue_lng='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            category_name=v['categories'][0]['name']
        except:
            category_name='no data'
        try:
            venue_id=v['id']
        except:
            venue_id='no data'
        
        restaurants_list.append([(
            venue_name, 
            venue_lat, 
            venue_lng, 
            category_name,
            venue_id)])

if len(breakfast_list)==0:
    print('No restaurants found within radius of {}m of {}'.format(radius, address))

else:
        restaurants_df = pd.DataFrame([item for venue_list in restaurants_list for item in venue_list])
        restaurants_df.columns = ['Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category','Venue ID']
        restaurants_df=restaurants_df[restaurants_df['Venue Category'].isin(restaurants_category_exclude)==False]
        restaurants_df.loc[:,'Venue Key']=range(len(restaurants_df.Venue))
        restaurants_df.reset_index(drop=True, inplace=True)
        print('Number of restaurants in {} (latitude: {},longitude: {}): {}\n'.format(address,latitude,longitude,restaurants_df.shape[0]))
        restaurants_df

Number of restaurants in Hamburg, Germany (latitude: 53.550341,longitude: 10.000654): 22



In [330]:
restaurants_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key
0,bona'me,53.548555,10.000944,Pide Place,58922d0618384c317754caeb,0
1,O-ren Ishii,53.548291,9.997890,Vietnamese Restaurant,4b151242f964a5202da823e3,1
2,PIZZAMACHER Trattoria,53.583087,9.975952,Italian Restaurant,5f7d9acc2214a218b5ebc8a2,2
3,Wandelhalle,53.553285,10.006006,Shopping Mall,4c2d048c987f20a1c9b748aa,3
4,Nom,53.549029,9.974900,Vietnamese Restaurant,59a41b3d83e3805cc2b0599d,4


#### 3.4.3 Add restaurants' ratings

In [331]:
# retrieve ratings of restaurants and append to restaurants_df (Attention: venue/rating is a premium call!)
venue_id=restaurants_df.loc[:,'Venue ID']
restaurants_df.loc[:,'Venue Rating']=0

for i in range(len(venue_id)):
#    print('\nvenue_id [',i,']:',venue_id[i])
    venue_var=venue_id[i]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_var, CLIENT_ID, CLIENT_SECRET, VERSION)
    try:
#        print('Venue rating: ', requests.get(url).json()['response']['venue']['rating'])
        restaurants_df.loc[i,'Venue Rating']=requests.get(url).json()['response']['venue']['rating']
    except:
#        print('Venue rating: This venue has not been rated yet.')
        restaurants_df.loc[i,'Venue Rating']='no rating'

restaurants_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating
0,bona'me,53.548555,10.000944,Pide Place,58922d0618384c317754caeb,0,no rating
1,O-ren Ishii,53.548291,9.997890,Vietnamese Restaurant,4b151242f964a5202da823e3,1,no rating
2,PIZZAMACHER Trattoria,53.583087,9.975952,Italian Restaurant,5f7d9acc2214a218b5ebc8a2,2,no rating
3,Wandelhalle,53.553285,10.006006,Shopping Mall,4c2d048c987f20a1c9b748aa,3,no rating
4,Nom,53.549029,9.974900,Vietnamese Restaurant,59a41b3d83e3805cc2b0599d,4,no rating


#### 3.4.4 Show top 100 rated restaurants (restaurants without rating dropped)

In [332]:
#top_restaurants_df=restaurants_df.loc[((restaurants_df['Venue Rating']!='no rating')&(restaurants_df['Venue Name'].str.contains('estaurant')==False)&(restaurants_df['Venue Name'].str.contains('tore')==False)&(restaurants_df['Venue Name'].str.contains('akery')==False)),:].sort_values('Venue Rating', ascending=False)
top_restaurants_df=restaurants_df.loc[((restaurants_df['Venue Rating']!='no rating')),:].sort_values('Venue Rating', ascending=False)
top_restaurants_df.reset_index(drop=True, inplace=True)
top_restaurants_df=top_restaurants_df.head(100)
top_restaurants_df

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating


#### 3.4.5 Determine 6 dining areas (restaurants clustered into 6 clusters based on coordinates)

3.4.5.1 Run _k_-means to cluster the restaurants into 6 clusters.


In [333]:
# set number of clusters
restaurants_kclusters = 6

dining_areas = restaurants_df.drop(['Venue','Venue Category', 'Venue ID', 'Venue Rating','Venue Key'], 1)

# run k-means clustering
restaurants_kmeans = KMeans(n_clusters=restaurants_kclusters, random_state=0).fit(dining_areas)

# check cluster labels generated for each row in the dataframe
restaurants_kmeans.labels_[0:10] 

array([5, 5, 0, 5, 3, 4, 3, 1, 2, 1], dtype=int32)

In [334]:
dining_areas.head()

,Venue Latitude,Venue Longitude
0,53.548555,10.000944
1,53.548291,9.997890
2,53.583087,9.975952
3,53.553285,10.006006
4,53.549029,9.974900


3.4.5.2 Create a dataframe that includes the dining areas and the restaurants in each dining area.


In [335]:
# add clustering labels
try:
    dining_areas.insert(0, 'Item Key', restaurants_kmeans.labels_)
    dining_areas['Venue Key']=restaurants_df['Venue Key']
    dining_areas_merged = restaurants_df
except:
    dining_areas_merged = restaurants_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
dining_areas_merged = dining_areas_merged.join(dining_areas[['Venue Key','Item Key']].set_index('Venue Key'), on='Venue Key')

dining_areas_merged.head() # check the last columns!

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating,Item Key
0,bona'me,53.548555,10.000944,Pide Place,58922d0618384c317754caeb,0,no rating,5
1,O-ren Ishii,53.548291,9.997890,Vietnamese Restaurant,4b151242f964a5202da823e3,1,no rating,5
2,PIZZAMACHER Trattoria,53.583087,9.975952,Italian Restaurant,5f7d9acc2214a218b5ebc8a2,2,no rating,0
3,Wandelhalle,53.553285,10.006006,Shopping Mall,4c2d048c987f20a1c9b748aa,3,no rating,5
4,Nom,53.549029,9.974900,Vietnamese Restaurant,59a41b3d83e3805cc2b0599d,4,no rating,3


3.4.5.3 Determine centroid coordinates of dining areas

In [336]:
# Centroid coordinates
for i in dining_areas_merged.index:
    dining_areas_merged.loc[i,'Item Latitude']=restaurants_kmeans.cluster_centers_[dining_areas_merged['Item Key'][i]][0]
    dining_areas_merged.loc[i,'Item Longitude']=restaurants_kmeans.cluster_centers_[dining_areas_merged['Item Key'][i]][1]
dining_areas_merged.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating,Item Key,Item Latitude,Item Longitude
0,bona'me,53.548555,10.000944,Pide Place,58922d0618384c317754caeb,0,no rating,5,53.551676,10.003806
1,O-ren Ishii,53.548291,9.997890,Vietnamese Restaurant,4b151242f964a5202da823e3,1,no rating,5,53.551676,10.003806
2,PIZZAMACHER Trattoria,53.583087,9.975952,Italian Restaurant,5f7d9acc2214a218b5ebc8a2,2,no rating,0,53.587790,9.982941
3,Wandelhalle,53.553285,10.006006,Shopping Mall,4c2d048c987f20a1c9b748aa,3,no rating,5,53.551676,10.003806
4,Nom,53.549029,9.974900,Vietnamese Restaurant,59a41b3d83e3805cc2b0599d,4,no rating,3,53.551974,9.966428


3.4.5.4 Show the 6 dining areas with their coordinates

In [337]:
top_dining_areas=dining_areas_merged.loc[:,['Item Key', 'Item Latitude','Item Longitude']].groupby('Item Key', as_index=False).mean()
top_dining_areas.loc[:,'Item Category']='Dining Area'
for i in top_dining_areas.loc[:,'Item Key']:
    top_dining_areas.loc[i,'Item Name']='Dining Area '+str(top_dining_areas.loc[i,'Item Key'])
top_dining_areas

,Item Key,Item Latitude,Item Longitude,Item Category,Item Name
0,0,53.587790,9.982941,Dining Area,Dining Area 0
1,1,53.546202,9.987125,Dining Area,Dining Area 1
2,2,53.557199,9.929676,Dining Area,Dining Area 2
3,3,53.551974,9.966428,Dining Area,Dining Area 3
4,4,53.554091,10.040635,Dining Area,Dining Area 4
5,5,53.551676,10.003806,Dining Area,Dining Area 5


#### 3.5 Search for areas with top rated nightlife venues

##### 3.5.1 Retrieve all bars of target city as a dataframe

In [338]:
# Foursquare.com credentials and parameters
CLIENT_ID = 'R2B3BSMOAZU414BSOGLAYMB2MJPFUDT1PPEL1M0KZMD4RJFK' # your Foursquare ID
CLIENT_SECRET = '1YM0O41P05UKBSIDUTBL1G4ZYCAKEVRNQ4VFIO40THDU3EWW' # your Foursquare Secret
VERSION = '20201110' # Foursquare API version

categoryId='4d4b7105d754a06376d81259,52e81612bcbc57f1066b79ec,52e81612bcbc57f1066b7a06'

#Nightlife Spot
#4d4b7105d754a06376d81259
#Salsa Club
#52e81612bcbc57f1066b79ec
#Irish Pub
#52e81612bcbc57f1066b7a06

bars_category_name_exclude=[]

radius=5000
LIMIT = 100 # A default Foursquare API limit value

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            latitude, 
            longitude, 
            VERSION, 
            radius,
            LIMIT,
            categoryId)
            
# make the GET request
bars_json = requests.get(url).json()['response']['venues']
       
# return only relevant information for each nearby venue
bars_list=[]


for v in bars_json:
        try:
            venue_name=v['name']
        except:
            venue_name='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            venue_lng=v['location']['lng']
        except:
            venue_lng='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            category_name=v['categories'][0]['name']
        except:
            category_name='no data'
        try:
            venue_id=v['id']
        except:
            venue_id='no data'
        
        bars_list.append([(
            venue_name, 
            venue_lat, 
            venue_lng, 
            category_name,
            venue_id)])

if len(bars_list)==0:
    print('No bars found within radius of {}m of {}'.format(radius, address))

else:
        bars_df = pd.DataFrame([item for venue_list in bars_list for item in venue_list])
        bars_df.columns = ['Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category','Venue ID']
        bars_df=bars_df[bars_df['Venue Category'].isin(bars_category_name_exclude)==False]
        bars_df.loc[:,'Venue Key']=range(len(bars_df.Venue))
        bars_df.reset_index(drop=True, inplace=True)
        print('Number of bars in {} (latitude: {},longitude: {}): {}\n'.format(address,latitude,longitude,bars_df.shape[0]))
        bars_df

Number of bars in Hamburg, Germany (latitude: 53.550341,longitude: 10.000654): 49



In [339]:
bars_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key
0,Hofbräu Wirtshaus am Speersort,53.549546,9.997839,Bavarian Restaurant,4e748542e4cd325436dfd642,0
1,ALEX,53.544466,9.975225,Café,5cc0a10b8afbe00032e2f90d,1
2,Kinfelts,53.541431,9.987667,Wine Bar,5b773ec0f2554e0039b3c2c0,2
3,Wohlers,53.557168,9.954764,Restaurant,4e076572d22d658532a5a5b7,3
4,Craft Bier Bar Hamburg,53.563400,9.958881,Beer Bar,5d8357cf615ed5000831d0e1,4


##### 3.5.2 Add bars' ratings

In [340]:
# retrieve ratings of bars and append to museums_df (Attention: venue/rating is a premium call!)
venue_id=bars_df.loc[:,'Venue ID']
bars_df.loc[:,'Venue Rating']=0

for i in range(len(venue_id)):
#    print('\nvenue_id [',i,']:',venue_id[i])
    venue_var=venue_id[i]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_var, CLIENT_ID, CLIENT_SECRET, VERSION)
    try:
#        print('Venue rating: ', requests.get(url).json()['response']['venue']['rating'])
        bars_df.loc[i,'Venue Rating']=requests.get(url).json()['response']['venue']['rating']
    except:
#        print('Venue rating: This venue has not been rated yet.')
        bars_df.loc[i,'Venue Rating']='no rating'

bars_df.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating
0,Hofbräu Wirtshaus am Speersort,53.549546,9.997839,Bavarian Restaurant,4e748542e4cd325436dfd642,0,no rating
1,ALEX,53.544466,9.975225,Café,5cc0a10b8afbe00032e2f90d,1,no rating
2,Kinfelts,53.541431,9.987667,Wine Bar,5b773ec0f2554e0039b3c2c0,2,no rating
3,Wohlers,53.557168,9.954764,Restaurant,4e076572d22d658532a5a5b7,3,no rating
4,Craft Bier Bar Hamburg,53.563400,9.958881,Beer Bar,5d8357cf615ed5000831d0e1,4,no rating


##### 3.5.3 Show top 5 rated bars (bars without rating dropped)

In [341]:
#top_bars_df=bars_df.loc[((bars_df['Venue Rating']!='no rating')&(bars_df['Venue Name'].str.contains('estaurant')==False)),:].sort_values('Venue Rating', ascending=False)
top_bars_df=bars_df.loc[((bars_df['Venue Rating']!='no rating')),:].sort_values('Venue Rating', ascending=False)
top_bars_df.reset_index(drop=True, inplace=True)
top_bars_df=top_bars_df.head(5)
top_bars_df

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating


##### 3.5.4 Determine 6 nightlife areas (bars clustered into 6 clusters based on coordinates)

3.5.4.1 Run _k_-means to cluster the bars into 6 clusters.


In [342]:
# set number of clusters
bars_kclusters = 6

bars_areas = bars_df.drop(['Venue','Venue Category', 'Venue ID', 'Venue Rating','Venue Key'], 1)

# run k-means clustering
bars_kmeans = KMeans(n_clusters=bars_kclusters, random_state=0).fit(bars_areas)

# check cluster labels generated for each row in the dataframe
bars_kmeans.labels_[0:10] 

array([1, 1, 1, 0, 0, 0, 4, 3, 1, 5], dtype=int32)

In [343]:
bars_areas.head()

,Venue Latitude,Venue Longitude
0,53.549546,9.997839
1,53.544466,9.975225
2,53.541431,9.987667
3,53.557168,9.954764
4,53.563400,9.958881


##### 3.5.4.2 Create a dataframe that shows the 6 nightlife areas and the bars in each nightlife area.


In [344]:
# add clustering labels
try:
    bars_areas.insert(0, 'Item Key', bars_kmeans.labels_)
    bars_areas['Venue Key']=bars_df['Venue Key']
    bars_areas_merged = bars_df
except:
    bars_areas_merged = bars_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
bars_areas_merged = bars_areas_merged.join(bars_areas[['Venue Key','Item Key']].set_index('Venue Key'), on='Venue Key')

bars_areas_merged.head() # check the last columns!

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating,Item Key
0,Hofbräu Wirtshaus am Speersort,53.549546,9.997839,Bavarian Restaurant,4e748542e4cd325436dfd642,0,no rating,1
1,ALEX,53.544466,9.975225,Café,5cc0a10b8afbe00032e2f90d,1,no rating,1
2,Kinfelts,53.541431,9.987667,Wine Bar,5b773ec0f2554e0039b3c2c0,2,no rating,1
3,Wohlers,53.557168,9.954764,Restaurant,4e076572d22d658532a5a5b7,3,no rating,0
4,Craft Bier Bar Hamburg,53.563400,9.958881,Beer Bar,5d8357cf615ed5000831d0e1,4,no rating,0


3.5.4.3 Determine centroid coordinates of nightlife areas

In [345]:
# Centroid coordinates of nightlife areas
for i in bars_areas_merged.index:
    bars_areas_merged.loc[i,'Item Latitude']=bars_kmeans.cluster_centers_[bars_areas_merged['Item Key'][i]][0]
    bars_areas_merged.loc[i,'Item Longitude']=bars_kmeans.cluster_centers_[bars_areas_merged['Item Key'][i]][1]
bars_areas_merged.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue ID,Venue Key,Venue Rating,Item Key,Item Latitude,Item Longitude
0,Hofbräu Wirtshaus am Speersort,53.549546,9.997839,Bavarian Restaurant,4e748542e4cd325436dfd642,0,no rating,1,53.549552,9.987616
1,ALEX,53.544466,9.975225,Café,5cc0a10b8afbe00032e2f90d,1,no rating,1,53.549552,9.987616
2,Kinfelts,53.541431,9.987667,Wine Bar,5b773ec0f2554e0039b3c2c0,2,no rating,1,53.549552,9.987616
3,Wohlers,53.557168,9.954764,Restaurant,4e076572d22d658532a5a5b7,3,no rating,0,53.553426,9.958502
4,Craft Bier Bar Hamburg,53.563400,9.958881,Beer Bar,5d8357cf615ed5000831d0e1,4,no rating,0,53.553426,9.958502


3.5.4.3 Show nightlife areas and their coordinates

In [346]:
top_nightlife_areas=bars_areas_merged.loc[:,['Item Key', 'Item Latitude','Item Longitude']].groupby('Item Key', as_index=False).mean()
top_nightlife_areas.loc[:,'Item Category']='Nightlife Area'
for i in top_nightlife_areas.loc[:,'Item Key']:
    top_nightlife_areas.loc[i,'Item Name']='Nightlife Area '+str(top_nightlife_areas.loc[i,'Item Key'])
top_nightlife_areas

,Item Key,Item Latitude,Item Longitude,Item Category,Item Name
0,0,53.553426,9.958502,Nightlife Area,Nightlife Area 0
1,1,53.549552,9.987616,Nightlife Area,Nightlife Area 1
2,2,53.529372,10.037802,Nightlife Area,Nightlife Area 2
3,3,53.558238,10.007056,Nightlife Area,Nightlife Area 3
4,4,53.500511,9.990703,Nightlife Area,Nightlife Area 4
5,5,53.593136,10.021813,Nightlife Area,Nightlife Area 5


#### 3.6 Determine 6 unique day-long itineraries

3.6.1 Generate dataframe with coordinates of top_breakfast_areas, top_museums, top_sights, top_dining_ares, top_nightlife_areas

In [347]:
day_df=pd.DataFrame()
day_df=top_breakfast_areas
day_df=day_df.append(top_museums_df[['Item Key', 'Item Name','Item Latitude', 'Item Longitude','Item Category']])
day_df=day_df.append(top_sights_df[['Item Key', 'Item Latitude', 'Item Longitude','Item Name','Item Category']])
day_df=day_df.append(top_dining_areas[['Item Key', 'Item Latitude', 'Item Longitude','Item Name','Item Category']])
day_df=day_df.append(top_nightlife_areas[['Item Key', 'Item Latitude', 'Item Longitude','Item Name','Item Category']])
day_df=day_df[['Item Category', 'Item Name','Item Key', 'Item Latitude', 'Item Longitude']]
day_df.reset_index(drop=True, inplace=True)
day_df.loc[:,'Item Key']=day_df.index
day_df

,Item Category,Item Name,Item Key,Item Latitude,Item Longitude
0,Breakfast Area,Breakfast Area 0,0,53.551651,9.993620
1,Breakfast Area,Breakfast Area 1,1,53.563466,9.941182
2,Breakfast Area,Breakfast Area 2,2,53.549519,10.037344
3,Breakfast Area,Breakfast Area 3,3,53.573069,10.004401
4,Breakfast Area,Breakfast Area 4,4,53.558838,9.966830
5,Breakfast Area,Breakfast Area 5,5,53.505420,9.986052
6,Dining Area,Dining Area 0,6,53.587790,9.982941
7,Dining Area,Dining Area 1,7,53.546202,9.987125
8,Dining Area,Dining Area 2,8,53.557199,9.929676
9,Dining Area,Dining Area 3,9,53.551974,9.966428


3.6.2 Run _k_-means to cluster the breakfast venues into 6 clusters.


In [348]:
# set number of clusters
day_kclusters = 6

day_areas = day_df.drop(['Item Category','Item Name'], 1)

# run k-means clustering
day_kmeans = KMeans(n_clusters=day_kclusters, random_state=0).fit(day_areas)

# check cluster labels generated for each row in the dataframe
day_kmeans.labels_[0:10] 

array([3, 3, 3, 1, 1, 1, 1, 5, 5, 5], dtype=int32)

In [349]:
day_areas.head()

,Item Key,Item Latitude,Item Longitude
0,0,53.551651,9.993620
1,1,53.563466,9.941182
2,2,53.549519,10.037344
3,3,53.573069,10.004401
4,4,53.558838,9.966830


3.6.3 Create a dataframe that includes the itineraries and the items in each itinerary.


In [350]:
# add clustering labels
try:
    day_areas.insert(0, 'Day Area', day_kmeans.labels_)
#    day_areas['Venue Key']=day_df['Venue Key']
    day_areas_merged = day_df
except:
    day_areas_merged = day_df

# merge day_areas_merged with day_areas to add latitude/longitude for each area
day_areas_merged = day_areas_merged.join(day_areas[['Item Key','Day Area']].set_index('Item Key'), on='Item Key')

day_areas_merged.head() # check the last columns!

,Item Category,Item Name,Item Key,Item Latitude,Item Longitude,Day Area
0,Breakfast Area,Breakfast Area 0,0,53.551651,9.993620,3
1,Breakfast Area,Breakfast Area 1,1,53.563466,9.941182,3
2,Breakfast Area,Breakfast Area 2,2,53.549519,10.037344,3
3,Breakfast Area,Breakfast Area 3,3,53.573069,10.004401,1
4,Breakfast Area,Breakfast Area 4,4,53.558838,9.966830,1


In [351]:
# Centroid coordinates of breakfast areas
for i in day_areas_merged.index:
    day_areas_merged.loc[i,'Day Area Latitude']=day_kmeans.cluster_centers_[day_areas_merged['Day Area'][i]][1]
    day_areas_merged.loc[i,'Day Area Longitude']=day_kmeans.cluster_centers_[day_areas_merged['Day Area'][i]][2]
day_areas_merged.head()

,Item Category,Item Name,Item Key,Item Latitude,Item Longitude,Day Area,Day Area Latitude,Day Area Longitude
0,Breakfast Area,Breakfast Area 0,0,53.551651,9.993620,3,53.554879,9.990715
1,Breakfast Area,Breakfast Area 1,1,53.563466,9.941182,3,53.554879,9.990715
2,Breakfast Area,Breakfast Area 2,2,53.549519,10.037344,3,53.554879,9.990715
3,Breakfast Area,Breakfast Area 3,3,53.573069,10.004401,1,53.556279,9.985056
4,Breakfast Area,Breakfast Area 4,4,53.558838,9.966830,1,53.556279,9.985056


3.6.4 Calculate distances between breakfast areas and top museums

In [352]:
import math
d1=pd.DataFrame()
for i in range(6):  # loop through breakfast areas
    for j in range(6): # loop through top museums
        d1.loc[i,j]=math.sqrt( (top_museums_df.loc[j,'Item Latitude']-top_breakfast_areas.loc[i,'Item Latitude'])**2 + (top_museums_df.loc[j,'Item Longitude']-top_breakfast_areas.loc[i,'Item Longitude'])**2)
d1=d1.set_index(top_breakfast_areas.loc[:,'Item Name'].unique())
d1.columns=top_museums_df.loc[:,'Item Name'].unique()
d1    

KeyError: 0

3.6.5 Calculate distances between top museums and top sights

In [ ]:
import math
d2=pd.DataFrame()
for i in range(6):  # loop through top museums
    for j in range(6): # loop through top sights
        d2.loc[i,j]=math.sqrt( (top_sights_df.loc[j,'Item Latitude']-top_museums_df.loc[i,'Item Latitude'])**2 + (top_sights_df.loc[j,'Item Longitude']-top_museums_df.loc[i,'Item Longitude'])**2)
d2=d2.set_index(top_museums_df.loc[:,'Item Name'].unique())
d2.columns=top_sights_df.loc[:,'Item Name'].unique()
d2    

3.6.6 Calculate distances between top sights and nearest dining area

In [ ]:
import math
d3=pd.DataFrame()
for i in range(6):  # loop through top museums
    for j in range(6): # loop through top sights
        d3.loc[i,j]=math.sqrt( (top_dining_areas.loc[j,'Item Latitude']-top_sights_df.loc[i,'Item Latitude'])**2 + (top_dining_areas.loc[j,'Item Longitude']-top_sights_df.loc[i,'Item Longitude'])**2)
d3=d3.set_index(top_sights_df.loc[:,'Item Name'].unique())
d3.columns=top_dining_areas.loc[:,'Item Name'].unique()
d3    

3.6.7 Calculate distances between dining area and nearest nightlife area

In [ ]:
import math
d4=pd.DataFrame()
for i in range(6):  # loop through top museums
    for j in range(6): # loop through top sights
        d4.loc[i,j]=math.sqrt( (top_nightlife_areas.loc[j,'Item Latitude']-top_dining_areas.loc[i,'Item Latitude'])**2 + (top_nightlife_areas.loc[j,'Item Longitude']-top_dining_areas.loc[i,'Item Longitude'])**2)
d4=d4.set_index(top_dining_areas.loc[:,'Item Name'].unique())
d4.columns=top_nightlife_areas.loc[:,'Item Name'].unique()
d4    

3.6.8 Generate dataframe with optimum itineraries

In [ ]:
day_trips_df=pd.DataFrame()
for i in range(6):
    day_trips_df.loc[i,'Breakfast Area']=d1.loc[d1.iloc[:,i]==d1.iloc[:,i].min(),d1.columns[i]].index[0]
    day_trips_df.loc[i,'Museum']=d1.columns[i]
    day_trips_df.loc[i,'Sight']=d2.T.loc[d2.T.iloc[:,i]==d2.T.iloc[:,i].min(),d2.T.columns[i]].index[0]
    day_trips_df.loc[i,'Dining Area']=d3.T.loc[d3.T.loc[:,d3.T.loc[:,d3.T.columns==day_trips_df.loc[i,'Sight']].min().index[0]]==d3.T.loc[:,d3.T.loc[:,d3.T.columns==day_trips_df.loc[i,'Sight']].min().index[0]].min(),d3.T.loc[:,d3.T.columns==day_trips_df.loc[i,'Sight']].min().index[0]].index[0]
    day_trips_df.loc[i,'Nightlife Area']=d4.T.loc[d4.T.loc[:,d4.T.loc[:,d4.T.columns==day_trips_df.loc[i,'Dining Area']].min().index[0]]==d4.T.loc[:,d4.T.loc[:,d4.T.columns==day_trips_df.loc[i,'Dining Area']].min().index[0]].min(),d4.T.loc[:,d4.T.columns==day_trips_df.loc[i,'Dining Area']].min().index[0]].index[0]
    if i<6:
        d1=d1.drop(d1.loc[d1.iloc[:,i]==d1.iloc[:,i].min(),d1.columns[i]].index[0],axis=0)
        d2=d2.drop(d2.T.loc[d2.T.iloc[:,i]==d2.T.iloc[:,i].min(),d2.T.columns[i]].index[0],axis=1)
        d3=d3.drop(d3.T.loc[d3.T.loc[:,d3.T.loc[:,d3.T.columns==day_trips_df.loc[i,'Sight']].min().index[0]]==d3.T.loc[:,d3.T.loc[:,d3.T.columns==day_trips_df.loc[i,'Sight']].min().index[0]].min(),d3.T.loc[:,d3.T.columns==day_trips_df.loc[i,'Sight']].min().index[0]].index[0],axis=1)
        d4=d4.drop(d4.T.loc[d4.T.loc[:,d4.T.loc[:,d4.T.columns==day_trips_df.loc[i,'Dining Area']].min().index[0]]==d4.T.loc[:,d4.T.loc[:,d4.T.columns==day_trips_df.loc[i,'Dining Area']].min().index[0]].min(),d4.T.loc[:,d4.T.columns==day_trips_df.loc[i,'Dining Area']].min().index[0]].index[0] ,axis=1)
    else:
        d1=d1
        d2=d2
        d3=d3
        d4=d4
top_day_trips=day_trips_df
top_day_trips

In [ ]:
itinerary=top_day_trips.T
itinerary

In [ ]:
itinerary_3=itinerary[0].append(itinerary[1]).append(itinerary[2])

itinerary_df=pd.DataFrame(itinerary_3)
itinerary_df.index=['Day 1','Day 1','Day 1','Day 1','Day 1','Day 2','Day 2','Day 2','Day 2','Day 2','Day 3','Day 3','Day 3','Day 3','Day 3',]
itinerary_df.columns=['Item Name']
itinerary_df=itinerary_df.join(day_areas_merged[['Item Name','Item Latitude','Item Longitude']].set_index('Item Name'), on='Item Name')
itinerary_df

3.6.9 Determine centroid of breakfast areas and nightlife areas (1st and last item of each itinerary should be close to hotel)

In [ ]:
# set number of clusters
top_day_trips_kclusters = 1

breakfast_nightlife_top_day_trips=top_day_trips.loc[:,'Breakfast Area'].to_list() + top_day_trips.loc[:,'Nightlife Area'].to_list()
#day_areas_merged_2=day_areas_merged.loc[((day_areas_merged['Item Name'].isin(breakfast_top_day_trips)) or (day_areas_merged['Item Name'].isin(nightlife_top_day_trips))),:]
day_areas_merged_2=day_areas_merged.loc[day_areas_merged['Item Name'].isin(breakfast_nightlife_top_day_trips)==True,['Item Key','Item Latitude','Item Longitude']]


#hotel_area = top_day_trips.drop(['Venue','Venue Category', 'Venue ID', 'Venue Rating','Venue Key'], 1)

# run k-means clustering
top_day_trips_kmeans = KMeans(n_clusters=top_day_trips_kclusters, random_state=0).fit(day_areas_merged_2)

# generate hotel location
hotel_location=[top_day_trips_kmeans.cluster_centers_[0][1],top_day_trips_kmeans.cluster_centers_[0][2]]
print('The best hotel location is around coordinates '+str(hotel_location))

3.6.10 Determnine the best hotel (central location for all 3 itineraries)

In [ ]:
# Foursquare.com credentials and parameters
CLIENT_ID = 'R2B3BSMOAZU414BSOGLAYMB2MJPFUDT1PPEL1M0KZMD4RJFK' # your Foursquare ID
CLIENT_SECRET = '1YM0O41P05UKBSIDUTBL1G4ZYCAKEVRNQ4VFIO40THDU3EWW' # your Foursquare Secret
VERSION = '20201110' # Foursquare API version

categoryId='4bf58dd8d48988d1fa931735'

#Hotel
#4bf58dd8d48988d1fa931735

hotels_category_name_exclude=[]

radius=5000
LIMIT = 100 # A default Foursquare API limit value

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            hotel_location[0], 
            hotel_location[1], 
            VERSION, 
            radius,
            LIMIT,
            categoryId)
            
# make the GET request
hotels_json = requests.get(url).json()['response']['venues']
       
# return only relevant information for each nearby venue
hotels_list=[]


for v in hotels_json:
        try:
            venue_name=v['name']
        except:
            venue_name='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            venue_lng=v['location']['lng']
        except:
            venue_lng='no data'
        try:
            venue_lat=v['location']['lat']
        except:
            venue_lat='no data'
        try:
            category_name=v['categories'][0]['name']
        except:
            category_name='no data'
        try:
            venue_id=v['id']
        except:
            venue_id='no data'
        
        hotels_list.append([(
            venue_name, 
            venue_lat, 
            venue_lng, 
            category_name,
            venue_id)])

if len(hotels_list)==0:
    print('No hotels found within radius of {}m of the optimum hotel location'.format(radius))

else:
        hotels_df = pd.DataFrame([item for venue_list in hotels_list for item in venue_list])
        hotels_df.columns = ['Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Name','Venue ID']
        hotels_df.reset_index(drop=True, inplace=True)
        hotels_df=hotels_df[hotels_df['Venue Name'].isin(hotels_category_name_exclude)==False]
        print('Number of hotels in {} (latitude: {},longitude: {}): {}\n'.format(address,hotel_location[0],hotel_location[1],hotels_df.shape[0]))
        hotels_df

In [ ]:
hotels_df.head()

In [ ]:
# retrieve ratings of hotels and append to hotels_df (Attention: venue/rating is a premium call!)
venue_id=hotels_df.loc[:,'Venue ID']
hotels_df.loc[:,'Venue Rating']=0

for i in range(len(venue_id)):
#    print('\nvenue_id [',i,']:',venue_id[i])
    venue_var=venue_id[i]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_var, CLIENT_ID, CLIENT_SECRET, VERSION)
    try:
#        print('Venue rating: ', requests.get(url).json()['response']['venue']['rating'])
        hotels_df.loc[i,'Venue Rating']=requests.get(url).json()['response']['venue']['rating']
    except:
#        print('This hotel has not been rated yet.')
        hotels_df.loc[i,'Venue Rating']='no rating'

hotels_df.head()

In [ ]:
#Show top rated hotel near most convenient hotel location (hotels without rating dropped)
top_hotels_df=hotels_df.loc[((hotels_df['Venue Rating']!='no rating')),:].sort_values('Venue Rating', ascending=False)
top_hotels_df=top_hotels_df.head(1)
top_hotels_df

In [ ]:
hotel_lat=top_hotels_df.iloc[0,1]
hotel_lng=top_hotels_df.iloc[0,2]
hotel_name=top_hotels_df.iloc[0,0]
print('Hotel coordinates: '+str(hotel_lat)+', '+str(hotel_lng))

3.6.11 Generate itineraries' path coordinates

In [ ]:
# Route for day 1
route1=pd.DataFrame(data={'Item Name':hotel_name, 'Item Latitude':hotel_lat,'Item Longitude':hotel_lng},index=['Day 1'])
route_1=route1.append(itinerary_df[itinerary_df.index=='Day 1'])
route_1=route_1.append(route1)
route_1

In [ ]:
# Route for day 2
route2=pd.DataFrame(data={'Item Name':hotel_name, 'Item Latitude':hotel_lat,'Item Longitude':hotel_lng},index=['Day 2'])
route_2=route2.append(itinerary_df[itinerary_df.index=='Day 2'])
route_2=route_2.append(route2)
route_2

In [ ]:
# Route for day 3
route3=pd.DataFrame(data={'Item Name':hotel_name, 'Item Latitude':hotel_lat,'Item Longitude':hotel_lng},index=['Day 3'])
route_3=route3.append(itinerary_df[itinerary_df.index=='Day 3'])
route_3=route_3.append(route3)
route_3

In [ ]:
route_1_lat_lng=[]
for i in range(len(route_1)):
    route_1_lat_lng=route_1_lat_lng+[[route_1.iloc[i,1],route_1.iloc[i,2]]]
route_1_lat_lng

In [ ]:
route_2_lat_lng=[]
for i in range(len(route_2)):
    route_2_lat_lng=route_2_lat_lng+[[route_2.iloc[i,1],route_2.iloc[i,2]]]
route_2_lat_lng

In [ ]:
route_3_lat_lng=[]
for i in range(len(route_3)):
    route_3_lat_lng=route_3_lat_lng+[[route_3.iloc[i,1],route_3.iloc[i,2]]]
route_3_lat_lng

####3.7 Visualize the itineraries on a map

In [ ]:
from folium.features import DivIcon

In [ ]:
# create map
map_dining_areas = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the restaurant clusters
restaurants_x = np.arange(restaurants_kclusters)
restaurants_ys = [i + restaurants_x + (i*restaurants_x)**2 for i in range(restaurants_kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(restaurants_ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
    
for area_restaurants_lat, area_restaurants_lon, area_restaurants_cluster in zip(top_dining_areas['Item Latitude'].unique(), top_dining_areas['Item Longitude'].unique(), top_dining_areas['Item Name'].unique()):
    area_restaurants_label = folium.Popup(' Dining Area ' + str(area_restaurants_cluster), parse_html=True)
    folium.CircleMarker(
        [area_restaurants_lat, area_restaurants_lon],
        radius=5,
        popup=area_restaurants_label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=1.0).add_to(map_dining_areas)   
    
# set color scheme for the nightlife clusters
bars_x = np.arange(bars_kclusters)
bars_ys = [i + bars_x + (i*bars_x)**2 for i in range(bars_kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(bars_ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for area_bars_lat, area_bars_lon, area_bars_cluster in zip(top_nightlife_areas['Item Latitude'].unique(), top_nightlife_areas['Item Longitude'].unique(), top_nightlife_areas['Item Name'].unique()):
    # Define marker label
    area_bars_label = folium.Popup(' Nightlife Area ' + str(area_bars_cluster), parse_html=True)

    # define square marker
    #icon_square = BeautifyIcon(icon_shape='rectangle-dot', border_color='blue', border_width=5,)
    
    # add marker to map
    folium.CircleMarker(
        [area_bars_lat, area_bars_lon],
        radius=5,
        popup=area_bars_label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=1.0).add_to(map_dining_areas)

for area_breakfast_lat, area_breakfast_lon, area_breakfast_cluster in zip(top_breakfast_areas['Item Latitude'].unique(), top_breakfast_areas['Item Longitude'].unique(), top_breakfast_areas['Item Name'].unique()):
    # Define marker label
    area_breakfast_label = folium.Popup(' Breakfast Area ' + str(area_breakfast_cluster), parse_html=True)

    # add marker to map
    folium.CircleMarker(
        [area_breakfast_lat, area_breakfast_lon],
        radius=5,
        popup=area_breakfast_label,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=1.0).add_to(map_dining_areas)
        
for top_museums_lat, top_museums_lon, top_museums_cluster in zip(top_museums_df['Item Latitude'].unique(), top_museums_df['Item Longitude'].unique(), top_museums_df['Venue'].unique()):
    # Define marker label
    top_museums_label = folium.Popup(' Top Museum ' + str(top_museums_cluster), parse_html=True)
    
    # add marker to map
    folium.CircleMarker(
        [top_museums_lat, top_museums_lon],
        radius=5,
        popup=top_museums_label,
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=1.0).add_to(map_dining_areas)
    
for top_sights_lat, top_sights_lon, top_sights_cluster in zip(top_sights_df['Item Latitude'].unique(), top_sights_df['Item Longitude'].unique(), top_sights_df['Venue'].unique()):
    # Define marker label
    top_sights_label = folium.Popup(' Top Sight ' + str(top_sights_cluster), parse_html=True)
    
    # add marker to map
    folium.CircleMarker(
        [top_sights_lat, top_sights_lon],
        radius=5,
        popup=top_sights_label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=1.0).add_to(map_dining_areas)
        
# Show route and items of Day 1
for day_1_lat, day_1_lon, day_1_name, i in zip(itinerary_df.loc[itinerary_df.index=='Day 1', 'Item Latitude'], itinerary_df.loc[itinerary_df.index=='Day 1', 'Item Longitude'], itinerary_df.loc[itinerary_df.index=='Day 1', 'Item Name'], range(1,len(itinerary_df.loc[itinerary_df.index=='Day 1', 'Item Name'])+1)):
    # Define marker label
    day_1_label = folium.Popup('Day 1:' + str(day_1_name), parse_html=True)
    # add markers to map
    folium.CircleMarker(
        [day_1_lat, day_1_lon],
        radius=10,
        popup=day_1_label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=1.0
    ).add_to(map_dining_areas)
    folium.Marker(
        [day_1_lat, day_1_lon], 
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(10,4.5),
        html='<div style="font-size: 10px; color : white;text-align:center">'+str(i)+'</div>',
        )).add_to(map_dining_areas)
folium.PolyLine(route_1_lat_lng, color='blue').add_to(map_dining_areas)

# Show route and items of Day 2
for day_2_lat, day_2_lon, day_2_name, i in zip(itinerary_df.loc[itinerary_df.index=='Day 2', 'Item Latitude'], itinerary_df.loc[itinerary_df.index=='Day 2', 'Item Longitude'], itinerary_df.loc[itinerary_df.index=='Day 2', 'Item Name'], range(1,len(itinerary_df.loc[itinerary_df.index=='Day 2', 'Item Name'])+1)):
    # Define marker label
    day_2_label = folium.Popup('Day 2:' + str(day_2_name), parse_html=True)
    # add markers to map
    folium.CircleMarker(
        [day_2_lat, day_2_lon],
        radius=10,
        popup=day_2_label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=1.0
    ).add_to(map_dining_areas)
    folium.Marker(
        [day_2_lat, day_2_lon], 
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(10,4.5),
        html='<div style="font-size: 10px; color : white;text-align:center">'+str(i)+'</div>',
        )).add_to(map_dining_areas)
folium.PolyLine(route_2_lat_lng, color='green').add_to(map_dining_areas)

# Show route and items of Day 3
for day_3_lat, day_3_lon, day_3_name, i in zip(itinerary_df.loc[itinerary_df.index=='Day 3', 'Item Latitude'], itinerary_df.loc[itinerary_df.index=='Day 3', 'Item Longitude'], itinerary_df.loc[itinerary_df.index=='Day 3', 'Item Name'], range(1,len(itinerary_df.loc[itinerary_df.index=='Day 3', 'Item Name'])+1)):
    # Define marker label
    day_3_label = folium.Popup('Day 3:' + str(day_3_name), parse_html=True)
    # add markers to map
    folium.CircleMarker(
        [day_3_lat, day_3_lon],
        radius=10,
        popup=day_3_label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=1.0
    ).add_to(map_dining_areas)
    folium.Marker(
        [day_3_lat, day_3_lon], 
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(10,4.5),
        html='<div style="font-size: 10px; color : white;text-align:center">'+str(i)+'</div>',
        )).add_to(map_dining_areas)
folium.PolyLine(route_3_lat_lng, color='red').add_to(map_dining_areas)
    
# add hotel marker to map
folium.CircleMarker(
        [hotel_lat, hotel_lng],
        radius=20,
        popup='Hotel Ritz',
        tooltip='Hotel',
        color='magenta',
        fill=True,
        fill_color='magenta',
        fill_opacity=1.0,
).add_to(map_dining_areas)
folium.Marker(
        [hotel_lat, hotel_lng], 
        icon=DivIcon(
        icon_size=(40,40),
        icon_anchor=(20,9),
        html='<div style="font-size: 12px; color : white;text-align:center">Hotel</div>',
        )).add_to(map_dining_areas)

map_dining_areas